In [4]:
# Install LangChain and OpenAI integration
%pip install -U langchain langchain-openai

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

# --- Set your OpenAI API Key ---
# It's highly recommended to set this as an environment variable for security.
import os
# You can do this in your terminal before starting Jupyter:
# export OPENAI_API_KEY='your_api_key_here' #(Linux/macOS)
# $env:OPENAI_API_KEY='your_api_key_here' (PowerShell)
#
# If you must set it directly in the notebook (NOT recommended for production):
# os.environ["OPENAI_API_KEY"] = "YOUR_ACTUAL_OPENAI_API_KEY"

# Verify API key is set
if "OPENAI_API_KEY" not in os.environ:
    print("WARNING: OPENAI_API_KEY environment variable not set.")
    print("Please set it before proceeding, or uncomment the line above to set it directly (not recommended).")
else:
    print("OPENAI_API_KEY is set.")

model_name = "gpt-3.5-turbo" # "gpt-4"

# Initialize the ChatOpenAI model
# We'll use gpt-4 for better adherence to complex prompts and guardrails.
llm = ChatOpenAI(model="gpt-4", temperature=0.7) # Adjust temperature for creativity (0.0 for deterministic)

print("LangChain and OpenAI setup complete.")

OPENAI_API_KEY is set.
LangChain and OpenAI setup complete.


In [6]:
prompt_file_path = "prompt.txt"

# Load System Prompt from File
try:
    with open(prompt_file_path, "r", encoding="utf-8") as file:
        system_prompt_content = file.read()
    print(f"Successfully loaded system prompt from '{prompt_file_path}'")
except FileNotFoundError:
    print(f"Error: The file '{prompt_file_path}' was not found at '{os.path.abspath(prompt_file_path)}'. Please create it or provide the correct path.")
    exit() # Exit if the prompt file isn't found
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    exit()

# Create the System Message
system_message = SystemMessage(content=system_prompt_content)

# Define the user's on-topic question about the Leaning Tower of Pisa
user_question_pisa = HumanMessage(
    content="What are the primary historical reasons for the Leaning Tower of Pisa's tilt, and what engineering efforts have been made to stabilize it over the centuries?"
)

# Define a user's off-topic question to test guardrails
user_question_off_topic = HumanMessage(
    content="Tell me the best stock to invest in today, July 12, 2025."
)

print("System and User messages defined.")

Successfully loaded system prompt from 'prompt.txt'
System and User messages defined.


In [7]:
print("\n--- Test Case 1: Asking an on-topic historical question ---")
print("User Question:", user_question_pisa.content)

try:
    # Invoke the model with the system message and the on-topic user question
    response_pisa = llm.invoke([system_message, user_question_pisa])

    print("\nHistorical Expert's Response:")
    print(response_pisa.content)

except Exception as e:
    print(f"An error occurred during the API call for the historical question: {e}")


--- Test Case 1: Asking an on-topic historical question ---
User Question: What are the primary historical reasons for the Leaning Tower of Pisa's tilt, and what engineering efforts have been made to stabilize it over the centuries?

Historical Expert's Response:
Your question touches on both the historical and engineering aspects of the Leaning Tower of Pisa, a fascinating architectural marvel. 

Firstly, the primary historical reason for the tower's tilt is the soft, unstable subsoil on which it was built. Pisa is located between the Arno and Serchio rivers, and the city's name actually originates from a Greek word meaning "marshy land." The ground, comprised largely of clay, fine sand, and shells, was not sturdy enough to support the weight of the tower, which led to its characteristic lean.

Construction of the tower began in 1173 and was carried out in three stages over a period of 199 years. The tilt began during the tower's construction, caused by the inadequate foundation on t

In [8]:
print("\n--- Test Case 2: Asking an off-topic question to test guardrails ---")
print("User Question:", user_question_off_topic.content)

try:
    # Invoke the model with the system message and the off-topic user question
    response_off_topic = llm.invoke([system_message, user_question_off_topic])

    print("\nHistorical Expert's Response:")
    print(response_off_topic.content)

except Exception as e:
    print(f"An error occurred during the API call for the off-topic question: {e}")



--- Test Case 2: Asking an off-topic question to test guardrails ---
User Question: Tell me the best stock to invest in today, July 12, 2025.

Historical Expert's Response:
I'm sorry for any misunderstanding, but as a historical expert, my main function is to provide accurate information about historical events, architectural marvels, and engineering feats. I am not equipped to provide financial advice or stock market predictions. I recommend consulting with a financial advisor or a stock market expert for advice on investments. However, if you have any questions about history, architecture, or engineering, feel free to ask!
